In [ ]:
!pip install -q nx-arangodb networkx adbnx_adapter pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which i

In [ ]:
!pip install -q nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com

In [ ]:
import pandas as pd
import networkx as nx
from arango import ArangoClient
from adbnx_adapter import ADBNX_Adapter, ADBNX_Controller
from datetime import datetime

In [ ]:
def load_crime_data(csv_file_path):
    """Load and preprocess crime data from a CSV file."""
    df = pd.read_csv(csv_file_path)
    df.columns = [col.strip() for col in df.columns]  # Clean column names
    return df.fillna('Unknown')  # Handle missing values

def create_knowledge_graph(df):
    """Create a MultiDiGraph from crime data with enhanced relationships."""
    G = nx.MultiDiGraph()

    for _, row in df.iterrows():
        # Create unique IDs for each entity
        crime_id = f"Crime_{row['RowID']}"
        location_id = f"Location_{row['Location'].replace(' ', '_')}"
        crime_type_id = f"CrimeType_{row['CrimeCode']}"
        person_id = f"Person_{row['RowID']}"  # Using RowID to make unique persons
        weapon_id = f"Weapon_{row['Weapon']}" if row['Weapon'] else None
        neighborhood_id = f"Neighborhood_{row['Neighborhood']}" if row['Neighborhood'] else "Neighborhood_Unknown"
        district_id = f"District_{row['New_District']}" if row['New_District'] else "District_Unknown"
        premise_id = f"Premise_{row['PremiseType'].strip()}" if row['PremiseType'] else "Premise_Unknown"

        # Parse datetime
        crime_datetime = row['CrimeDateTime']
        try:
            dt_obj = datetime.strptime(crime_datetime, "%m/%d/%Y %I:%M:%S %p")
            hour_of_day = dt_obj.hour
            day_of_week = dt_obj.strftime('%A')
            month = dt_obj.strftime('%B')
            year = dt_obj.year
        except (ValueError, TypeError):
            hour_of_day = None
            day_of_week = None
            month = None
            year = None

        # Add nodes with attributes
        G.add_node(crime_id, type='Crime', datetime=crime_datetime, hour_of_day=hour_of_day, day_of_week=day_of_week, month=month, year=year)
        G.add_node(location_id, type='Location', latitude=row['Latitude'], longitude=row['Longitude'])
        G.add_node(crime_type_id, type='CrimeType', code=row['CrimeCode'], description=row['Description'])
        G.add_node(person_id, type='Person', gender=row['Gender'], age=row['Age'], race=row['Race'], ethnicity=row['Ethnicity'])
        G.add_node(neighborhood_id, type='Neighborhood', name=row['Neighborhood'])
        G.add_node(district_id, type='District', name=row['New_District'])
        G.add_node(premise_id, type='PremiseType', name=row['PremiseType'])

        if weapon_id:
            G.add_node(weapon_id, type='Weapon', name=row['Weapon'])
            G.add_edge(crime_id, weapon_id, relationship='USED')

        # Add edges with relationship types
        G.add_edge(crime_id, location_id, relationship='OCCURRED_AT')
        G.add_edge(crime_id, crime_type_id, relationship='CLASSIFIED_AS')
        G.add_edge(crime_id, person_id, relationship='INVOLVED')
        G.add_edge(location_id, neighborhood_id, relationship='LOCATED_IN')
        G.add_edge(neighborhood_id, district_id, relationship='PART_OF')
        G.add_edge(crime_id, premise_id, relationship='OCCURRED_IN')

    print(f"Created knowledge graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G

class CrimeGraphController(ADBNX_Controller):
    """Custom controller for crime graph translation."""
    def _identify_networkx_node(self, nx_node_id, nx_node, adb_v_cols):
        """Map NetworkX nodes to ArangoDB collections based on type."""
        return nx_node.get('type', 'Unknown')

    def _identify_networkx_edge(self, nx_edge, from_id, to_id, nx_map, adb_e_cols):
        """Map NetworkX edges to ArangoDB collections based on relationship."""
        return nx_edge.get('relationship', 'RELATED_TO')

In [ ]:
def main():
    # Load data and create graph
    df = load_crime_data("NIBRS.csv")
    nx_graph = create_knowledge_graph(df)

    # ArangoDB connection
    client = ArangoClient(hosts="https://d6b0bbc5c6ad.arangodb.cloud:8529")
    db = client.db("_system", username="root", password="cnA4t2mc33IS5LiC0YJI")

    # Define edge collections based on relationship types
    edge_definitions = [
        {"edge_collection": "OCCURRED_AT", "from_vertex_collections": ["Crime"], "to_vertex_collections": ["Location"]},
        {"edge_collection": "CLASSIFIED_AS", "from_vertex_collections": ["Crime"], "to_vertex_collections": ["CrimeType"]},
        {"edge_collection": "INVOLVED", "from_vertex_collections": ["Crime"], "to_vertex_collections": ["Person"]},
        {"edge_collection": "LOCATED_IN", "from_vertex_collections": ["Location"], "to_vertex_collections": ["Neighborhood"]},
        {"edge_collection": "PART_OF", "from_vertex_collections": ["Neighborhood"], "to_vertex_collections": ["District"]},
        {"edge_collection": "OCCURRED_IN", "from_vertex_collections": ["Crime"], "to_vertex_collections": ["PremiseType"]},
        {"edge_collection": "USED", "from_vertex_collections": ["Crime"], "to_vertex_collections": ["Weapon"]},
    ]

    # Create adapter and load to ArangoDB
    adapter = ADBNX_Adapter(db, CrimeGraphController())
    adapter.networkx_to_arangodb(
        "CrimeKnowledgeGraph",
        nx_graph,
        edge_definitions=edge_definitions,
        overwrite=True
    )
    print("Crime knowledge graph successfully loaded into ArangoDB.")

In [ ]:
main()

<ipython-input-8-f1e649f9400c>:3: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
[2025/03/07 11:08:39 +0000] [1111] [INFO] - adbnx_adapter: Instantiated ADBNX_Adapter with database '_system'
INFO:adbnx_adapter:Instantiated ADBNX_Adapter with database '_system'


Created knowledge graph with 102392 nodes and 321951 edges


Output()

Output()

[2025/03/07 11:09:19 +0000] [1111] [INFO] - adbnx_adapter: Created ArangoDB 'CrimeKnowledgeGraph' Graph
INFO:adbnx_adapter:Created ArangoDB 'CrimeKnowledgeGraph' Graph


Crime knowledge graph successfully loaded into ArangoDB.


In [ ]:
import nx_arangodb as nxadb
from arango import ArangoClient

[03:07:19 +0000] [INFO]: NetworkX-cuGraph is available.
INFO:nx_arangodb:NetworkX-cuGraph is available.


In [ ]:
db = ArangoClient(hosts="https://d6b0bbc5c6ad.arangodb.cloud:8529").db(username="root", password="cnA4t2mc33IS5LiC0YJI", verify=True)

print(db)



<StandardDatabase _system>


In [ ]:
G_adb = nxadb.MultiGraph(name="CrimeKnowledgeGraph", db=db, default_node_type=None)

[03:07:20 +0000] [INFO]: Graph 'CrimeKnowledgeGraph' exists.
INFO:nx_arangodb:Graph 'CrimeKnowledgeGraph' exists.
[03:07:20 +0000] [INFO]: Default node type set to 'Crime'
INFO:nx_arangodb:Default node type set to 'Crime'


In [ ]:
!pip install -q langchain langgraph langchain_openai langchain_anthropic langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
from langchain_community.graphs import ArangoGraph

arango_graph = ArangoGraph(db)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "SHERLOCK"

In [ ]:
model = ChatAnthropic(model="claude-3-7-sonnet-latest")

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from pydantic import BaseModel
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from typing_extensions import Literal
from langgraph.checkpoint.memory import MemorySaver
import re

In [ ]:
@tool
def transfer_to_profiling_expert():
  """Ask Profiling agent for help
      - Analyzes demographic and behavioral patterns
      - Creates suspect/victim profiles based on historical data
      - Identifies recurring patterns in criminal activity
  """
  return

@tool
def transfer_to_reporting_expert():
  """Ask reporting Agent for Help
      - Summarizes findings in structured reports
  """
  return

@tool
def tranfer_to_network_analysis_expert():
  """Ask Network Analysis for report
      - Specializes in identifying patterns and connections using NetworkX Algorithms
  """
@tool
def transfer_to_network_visualization():
  """ Use this tool to visualize network provide sample question to fetch subgraph and visualize""
  """
  return

supervisor_tools=[transfer_to_profiling_expert,tranfer_to_network_analysis_expert]

In [ ]:
from typing import List, Union, Optional, Dict, Any
from pydantic import BaseModel, Field


class NodeData(BaseModel):
    id: int = Field(description="unique id of the node")
    label: str = Field(description="label of the node")
    name: str = Field(description="name of the node to be displayed in the visialization")
    content: Optional[str] = None


class Node(BaseModel):
    data: NodeData = Field(description="data of the node")


class EdgeData(BaseModel):
    id: int = Field(description="unique id of the edge")
    label: str = Field(description="label of the edge")
    source: int  = Field(description="source node id")
    target: int = Field(description="target node id")


class Edge(BaseModel):
    data: EdgeData = Field(description="data of the edge")


class GraphElements(BaseModel):
    nodes: List[Node]
    edges: List[Edge]

In [ ]:
def supervisor(state: MessagesState) ->Command[Literal["network_expert", "profile_expert", "reporting_expert","__end__"]]:
    system_prompt = (
        """
        You are a coordinator expert for crime reporting tool called SHERLOCK ( Strategic Heuristic Engine for Research, Linking & Operational Crime Knowledge )
        Your task is to handle handoffs to various experts to answer user questions. If question is out of domain let the user know what you can do for them and do not mention you are coodinator or supervisor. Let you answer be precise.
        """
    )

    messages = [{"role": "system", "content": system_prompt}] + state["messages"]

    ai_msg = model.bind_tools(supervisor_tools).invoke(messages)


    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_profiling_expert":
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }

        return Command(
            goto="profile_expert", update={"messages": [ai_msg, tool_msg]}
        )

    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "tranfer_to_network_analysis_expert":
        tool_call_id = ai_msg.tool_calls[-1]["id"]

        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }

        return Command(
            goto="network_expert", update={"messages": [ai_msg, tool_msg]}
        )

    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_reporting_expert":
        tool_call_id = ai_msg.tool_calls[-1]["id"]

        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }

        return Command(
            goto="reporting_expert", update={"messages": [ai_msg, tool_msg]}
        )

    return {"messages": [ai_msg]}




In [ ]:
@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    ######################
    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()

    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    attempt = 1
    MAX_ATTEMPTS = 3
    text_to_nx_final = text_to_nx_cleaned

    while attempt <= MAX_ATTEMPTS:
        try:
            exec(text_to_nx_final, global_vars, local_vars)
            print(f"Successfully executed on attempt {attempt}")
            break  # Success, exit the loop
        except Exception as e:
            error_message = str(e)
            print(f"EXEC ERROR (Attempt {attempt}/{MAX_ATTEMPTS}): {error_message}")

            if attempt == MAX_ATTEMPTS:
                return f"Failed after {MAX_ATTEMPTS} attempts. Last error: {error_message}"

            # Try to fix the code with LLM
            print(f"Attempting to fix code (Attempt {attempt}/{MAX_ATTEMPTS})")

            correction_prompt = f"""
            I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

            I have the following graph analysis query: {query}.

            I tried to execute the following Python code to analyze a NetworkX graph G_adb:

            ```python
            {text_to_nx_final}
            ```

            But I got this error: {error_message}

            Please fix the code. Remember:
            1. The graph is called G_adb
            2. Only use networkx (as nx) and base Python libraries
            3. The final answer should be stored in a variable called FINAL_RESULT
            4. Only provide executable Python code, no explanations

            Your corrected code:
            """

            try:
                fixed_code_response = llm.invoke(correction_prompt).content
                # Clean up the response in case it returns markdown code blocks
                text_to_nx_final = re.sub(r"^```python\n|```$", "", fixed_code_response, flags=re.MULTILINE).strip()

                print('-'*10)
                print("FIXED CODE ATTEMPT:")
                print(text_to_nx_final)
                print('-'*10)
            except Exception as llm_error:
                print(f"Error requesting code correction: {llm_error}")
                return f"Failed to get code correction: {llm_error}"

            attempt += 1

    # If we exited the loop without a successful execution
    if attempt > MAX_ATTEMPTS:
        return f"All {MAX_ATTEMPTS} attempts failed. Could not execute the NetworkX algorithm."

    print('-'*10)
    if "FINAL_RESULT" not in local_vars:
        return "Execution succeeded but FINAL_RESULT variable was not set."

    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.

        Your response:
    """).content

    return nx_to_text

network_tools = [text_to_nx_algorithm_to_text,transfer_to_profiling_expert, transfer_to_reporting_expert]

In [ ]:
def network_expert(state:MessagesState)->Command[Literal["profile_expert", "reporting_expert","network_expert", "visualize_network","__end__"]]:
    system_prompt = (
        """
        You expert for crime reporting tool called SHERLOCK ( Strategic Heuristic Engine for Research, Linking & Operational Crime Knowledge )
        Your task is to Analyze networks using the text_to_nx_algorithm_to_text tool and if question reequires futher analysis transfer. When necessary to view subgraph of the data tranfer to visualize_network after analysis.

        """
    )

    messages = [{"role": "system", "content": system_prompt}] + state["messages"]

    ai_msg = model.bind_tools(network_tools).invoke(messages)

    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_profiling_expert":
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }

        return Command(
            goto="profile_expert", update={"messages": [ai_msg, tool_msg]}
        )


    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_reporting_expert":
        tool_call_id = ai_msg.tool_calls[-1]["id"]

        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }

        return Command(
            goto="reporting_expert", update={"messages": [ai_msg, tool_msg]}
        )

    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "text_to_nx_algorithm_to_text":
        tool_call_id = ai_msg.tool_calls[-1]["id"]

        content = text_to_nx_algorithm_to_text(ai_msg.tool_calls[-1].get("args").get("query"))
        tool_msg = {
            "role": "tool",
            "content": content,
            "tool_call_id": tool_call_id,
        }

        return Command(
            goto="network_expert", update={"messages":[ai_msg, tool_msg]}
        )

    if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_network_visualization":
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(
            goto="visualize_network", update={"messages": [ai_msg, tool_msg]}
        )

    return {"messages": [ai_msg]}

In [ ]:
from typing import Any
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
      allow_dangerous_requests=True,
      return_aql_query=True
    )

    result = chain.invoke(query)

    return str(result["result"])

profile_expert_tools = [text_to_aql_to_text, transfer_to_reporting_expert]

In [ ]:
def profile_expert(state:MessagesState)->Command[Literal["profile_expert","reporting_expert", "visualize_network","__end__"]]:
  system_prompt = (
      f"""
        You are an expert for crime reporting tool called SHERLOCK ( Strategic Heuristic Engine for Research, Linking & Operational Crime Knowledge )
        Your task is to the arango database use arango database to analyze and answer questions about crime using the tools provided and transfer when neccesary.
        When necessary to view subgraph of the data tranfer to visualize_network after analysis.

      """
  )

  ai_msg = model.bind_tools(profile_expert_tools).invoke([{"role":"system", "content":system_prompt}]+state["messages"])

  if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_reporting_expert":
      tool_call_id = ai_msg.tool_calls[-1]["id"]
      tool_msg = {
          "role": "tool",
          "content": "Successfully transferred",
          "tool_call_id": tool_call_id,
      }
      return Command(
          goto="reporting_expert", update={"messages": [ai_msg, tool_msg]}
      )

  if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "text_to_aql_to_text":
      tool_call_id = ai_msg.tool_calls[-1]["id"]
      content = text_to_aql_to_text(ai_msg.tool_calls[-1].get("args").get("query"))
      tool_msg = {
          "role": "tool",
          "content": content,
          "tool_call_id": tool_call_id,
      }

      return Command(
          goto="profile_expert", update={"messages":[ai_msg, tool_msg]}
      )

  if len(ai_msg.tool_calls) > 0 and ai_msg.tool_calls[-1].get("name") == "transfer_to_network_visualization":
      tool_call_id = ai_msg.tool_calls[-1]["id"]
      tool_msg = {
          "role": "tool",
          "content": "Successfully transferred",
          "tool_call_id": tool_call_id,
      }
      return Command(
          goto="visualize_network", update={"messages": [ai_msg, tool_msg]}
      )

  return {"messages":state["messages"]+ [{"role":"ai", "content":ai_msg.content}]}
def reporting_expert(state:MessagesState)->Command[Literal["__end__"]]:

  system_prompt = (
      """
        You are an expert in  crime reporting. Analyze our conversations and do a comprehensive report.

        Here is the conversation so far:
      """
  )

  messages = [{"role": "system", "content": system_prompt}] + state["messages"]

  ai_msg = model.invoke(messages)
  return {"messages": [ai_msg]}


In [ ]:
def visualize_graph(state:MessagesState)->Command[Literal["__end__"]]:

  system_prompt = (
      f"""
        You are an expert creating AQL query that follows a specific format for returning results for visualization later.

        Here is the arango schema {arango_graph.schema}
        Alway follow the format for the return query: {GraphElements}

      Guidelines for creating AQL:
        - Think step by step
        - Analyze user question and determine the best AQL query to answer the question.
        - Rely on `ArangoDB Schema` and `AQL Query Examples` (if provided) to generate the query
        - Rely on `ArangoDB Schema` and `AQL Query Examples` (if provided) to generate the query
        - Begin the `AQL Query` by the `WITH` AQL keyword to specify all of the ArangoDB Collections required
        - Use only the provided relationship types and properties in the `ArangoDB Schema` and any `AQL Query Examples` queries
        - Always use the `LIKE` operator for string matching
        - Structure the return value of the AQL Query in the a way that is easy to read and understand and answer the user's question  ,
        - Identify nodes and edges that might be relevant to the  question and include them in the AQL Query.

        Your query:
      """
  )

  ai_msg = model.invoke([{"role":"system", "content":system_prompt}]+state["messages"])

  result = arango_graph.query(ai_msg.content)
  data = GraphElements(nodes=result["nodes"], edges=result["edges"])
  return {"visualization_data": visualize_graph}

In [ ]:


class State(MessagesState):
  visualization_data:GraphElements

workflow = StateGraph(State)


workflow.add_node("supervisor", supervisor)
workflow.add_node("network_expert", network_expert)
workflow.add_node("profile_expert", profile_expert)
workflow.add_node("reporting_expert", reporting_expert)
workflow.add_node("visualize_network", visualize_graph)

workflow.add_edge(START, "supervisor")

checkpointer=MemorySaver()

graph = workflow.compile()

In [ ]:
from langchain_core.messages import convert_to_messages


def pretty_print_messages(update):
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")

    for node_name, node_update in update.items():
        print(f"Update from node {node_name}:")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")

In [ ]:
for chunk in graph.stream(
    {"messages": [("human", "Are there demographic patterns among victims of specific crime types? do a visualization")]},
):
    pretty_print_messages(chunk)

Update from node supervisor:


================================== Ai Message ==================================

[{'text': 'I can help you analyze demographic patterns among victims of specific crime types. This type of analysis falls under our profiling expertise, which examines demographic patterns and victimology across different crime categories.\n\nLet me transfer your request to our profiling expert who can provide this analysis and create visualizations of the demographic patterns:', 'type': 'text'}, {'id': 'toolu_01Pwpo3PjkqA9LNh6Ub9DQNz', 'input': {}, 'name': 'transfer_to_profiling_expert', 'type': 'tool_use'}]
Tool Calls:
  transfer_to_profiling_expert (toolu_01Pwpo3PjkqA9LNh6Ub9DQNz)
 Call ID: toolu_01Pwpo3PjkqA9LNh6Ub9DQNz
  Args:
================================= Tool Message =================================

Successfully transferred




> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Crime, Person, CrimeType, CLASSIFIED_AS, INVOLVED
FOR crime IN Crime
    

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-334-ae87ebbdd0bc>", line 1, in <cell line: 0>
    for chunk in graph.stream(
  File "/usr/local/lib/python3.11/dist-packages/langgraph/pregel/__init__.py", line 2024, in stream
    for _ in runner.tick(
  File "/usr/local/lib/python3.11/dist-packages/langgraph/pregel/runner.py", line 230, in tick
    run_with_retry(
  File "/usr/local/lib/python3.11/dist-packages/langgraph/pregel/retry.py", line 40, in run_with_retry
    return task.proc.invoke(task.input, config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langgraph/utils/runnable.py", line 546, in invoke
    input = step.invoke(input, config, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langgraph/utils/runna

TypeError: object of type 'NoneType' has no len()

In [33]:
for chunk in graph.stream(
    {"messages": [("human", "Are there demographic patterns among victims of specific crime types?")]},
      stream_mode=["updates", "messages"]
    ):
    print(chunk)

('messages', (AIMessageChunk(content=[], additional_kwargs={}, response_metadata={'model_name': 'claude-3-7-sonnet-20250219'}, id='run-d7a0848e-6bdf-4fd1-842f-45f10e445aad', usage_metadata={'input_tokens': 558, 'output_tokens': 1, 'total_tokens': 559, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_path': ('__pregel_pull', 'supervisor'), 'langgraph_checkpoint_ns': 'supervisor:a296f176-eda2-841e-f840-c2c8e4c37b4b', 'checkpoint_ns': 'supervisor:a296f176-eda2-841e-f840-c2c8e4c37b4b', 'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-7-sonnet-latest', 'ls_model_type': 'chat', 'ls_temperature': None, 'ls_max_tokens': 1024}))
('messages', (AIMessageChunk(content=[{'text': 'To', 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={}, id='run-d7a0848e-6bdf-4fd1-842f-45f10e445aad'), {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_t

ReadTimeout: HTTPSConnectionPool(host='d6b0bbc5c6ad.arangodb.cloud', port=8529): Read timed out. (read timeout=60)

In [ ]:
arango_graph.query("""WITH Crime, CrimeType, Person, INVOLVED, CLASSIFIED_AS
FOR crime IN Crime
    FOR classified IN CLASSIFIED_AS
        FILTER classified._from == crime._id
        FOR crimeType IN CrimeType
            FILTER classified._to == crimeType._id
            FILTER crimeType.description == "AUTO THEFT"
            FOR involved IN INVOLVED
                FILTER involved._from == crime._id
                FOR person IN Person
                    FILTER involved._to == person._id
                    COLLECT ageGroup = FLOOR(person.age / 10) * 10 INTO ageGroups
                    RETURN { ageGroup: ageGroup, count: LENGTH(ageGroups) }""")

[{'ageGroup': 0, 'count': 350},
 {'ageGroup': 10, 'count': 39},
 {'ageGroup': 20, 'count': 1050},
 {'ageGroup': 30, 'count': 1263},
 {'ageGroup': 40, 'count': 770},
 {'ageGroup': 50, 'count': 629},
 {'ageGroup': 60, 'count': 505},
 {'ageGroup': 70, 'count': 203},
 {'ageGroup': 80, 'count': 66},
 {'ageGroup': 90, 'count': 5}]